In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np

## 1. Get the data

You can download the data from Kaggle using [this link](https://www.kaggle.com/kaggle/us-baby-names/downloads/us-baby-names-release-2015-12-18-00-53-48.zip). It'll ask you to sign in to Kaggle after you click it. It's free to get an account if you don't have one. Just make sure the `read_csv()` function below points to the `StateNames.csv` file you downloaded.

In [3]:
df_full = pd.read_csv('StateNames.csv', dtype={'Year':str})
df_full.head()

,Id,Name,Year,Gender,State,Count
0,1,Mary,1910,F,AK,14
1,2,Annie,1910,F,AK,12
2,3,Anna,1910,F,AK,10
3,4,Margaret,1910,F,AK,8
4,5,Helen,1910,F,AK,7


## 2. Get rid of Washington, D.C.

Sorry DC, no representation for you, but at least there's no taxation involved here.

In [4]:
df_full = df_full[df_full['State'] != 'DC']

## 3. Map Gender and State to integers

I use -1/+1 for M/F, and I use the id numbers that go with each state in the US map's JSON data structure instead of the two letter state abbreviation. These id numbers are stored in `state_ids.csv` with columns "Abbrev" and "ID".

In [5]:
state_abbrevs = pd.read_csv('state_ids.csv', index_col=0, dtype={'ID':str})
gender_map = pd.Series([1, -1], index=['F', 'M'])
df_full['State'] = df_full['State'].map(state_abbrevs['ID'])
assert( ~np.any(pd.isnull(df_full['State'])) ) # make sure all states were accounted for
df_full['Gender'] = df_full['Gender'].map(gender_map)
df_full.head()

,Id,Name,Year,Gender,State,Count
0,1,Mary,1910,1,2,14
1,2,Annie,1910,1,2,12
2,3,Anna,1910,1,2,10
3,4,Margaret,1910,1,2,8
4,5,Helen,1910,1,2,7


## 4. Select only the most popular names

Sorry Zyshonne, Gelacio, and Geetika. There are a lot of names, and covering all of them would slow down things down when switching between names in the final visualization. Also, the less popular names have worse statistics, so visualizations would be less informative and less interesting. I use the total number of babies recorded with any particular name as the (maybe overly) simple metric for popularity.

In [6]:
num_names = 1000
name_by_popularity = df_full.pivot_table(index='Name', values='Count', aggfunc=np.sum)
name_by_popularity = pd.DataFrame(name_by_popularity).sort(['Count'], ascending=False)
name_by_popularity = name_by_popularity[:num_names]
name_by_popularity.head()

,Count
Name,
James,4924520
John,4811223
Robert,4696229
Michael,4283621
William,3810635


In [7]:
df_full = df_full[df_full['Name'].isin(name_by_popularity.index)]
assert(len(df_full['Name'].unique()) == num_names)
df_full.head()

,Id,Name,Year,Gender,State,Count
0,1,Mary,1910,1,2,14
1,2,Annie,1910,1,2,12
2,3,Anna,1910,1,2,10
3,4,Margaret,1910,1,2,8
4,5,Helen,1910,1,2,7


## 5. Generate reference pivot tables

I want one pivot table for males and one for females that contain the total number of each gender for a given year/state pair. Later, I'll generate similar pivot tables for each name, and these will be used as denominators to get the percent of females or males in any given state and in any given year who had any given name.

In [8]:
def pivot(df):
    """Given a dataframe with columns named Count, State, and Year, returns a
    pivot table data frame where rows are indexed by state, columns represent
    each year found in the Year data series, and numbers are combined using
    the numpy.sum function
    
    Parameters
    ----------
    df : pandas dataframe
    """
    return df.pivot_table(values='Count', index='State', columns='Year', aggfunc=np.sum, fill_value=0)

In [9]:
df_full_male   = df_full[df_full['Gender'] == gender_map['M']]
df_full_female = df_full[df_full['Gender'] == gender_map['F']]
df_full_male.head()

,Id,Name,Year,Gender,State,Count
13753,13754,John,1910,-1,2,8
13754,13755,James,1910,-1,2,7
13755,13756,Paul,1910,-1,2,6
13756,13757,Robert,1910,-1,2,6
13757,13758,Carl,1910,-1,2,5


In [10]:
pt_full_male = pivot(df_full_male)
pt_full_female = pivot(df_full_female)
pt_full_female.head()

Year,1910,1911,1912,1913,1914,1915,1916,1917,1918,1919,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
State,,,,,,,,,,,,,,,,,,,,,
1,10554,9779,12740,13527,16212,18659,20556,20161,21716,24727,...,14173,14343,14651,14027,13069,11951,11562,10999,11057,11067
10,489,526,764,807,1168,1582,1601,1773,2073,2055,...,2769,2783,2622,2526,2357,2233,2261,2091,1948,2070
12,3516,3298,4702,4874,6348,7297,7547,7727,8477,8706,...,57109,57737,56681,53385,49718,47301,46382,45570,45827,46090
13,12656,12313,16241,16810,20316,23388,25524,25487,27985,30354,...,33757,34629,34212,32431,30369,28271,27559,26966,26330,26595
15,341,423,631,742,964,1445,1720,1885,1904,1924,...,2383,2506,2344,2403,2253,2190,2120,2207,2124,2128


## 6. Generate a CSV file for each name

For each year/state/name combo, I want to know whether the name was "more" female or male, meaning a higher percentage of females vs males polled from that state that year had that name. This determines if it's on the pink or blue scale. If it's on the pink scale, the color will be determined by the percentage of females with that name that year, scaled by the greatest percentage of females from any state in any year (or the 95th percentile of all of those values). If it's on the blue scale, replace male with female in that last sentence.

In [11]:
grouped = df_full.groupby(['Name'])

In [12]:
name_list = []
percent_list = []
for i, (name, df_name) in enumerate(grouped):
    df_name_male   = df_name[df_name['Gender'] == gender_map['M']]
    df_name_female = df_name[df_name['Gender'] == gender_map['F']]
    pt_name_male   = pivot(df_name_male).divide(pt_full_male, fill_value=0)
    pt_name_female = pivot(df_name_female).divide(pt_full_female, fill_value=0)
    pt_combine = pt_name_female.where(pt_name_female>pt_name_male, pt_name_male * -1)
    scale = np.percentile(np.abs(pt_combine.values), 95)
    name_list.append(name)
    percent_list.append(scale * 100)
    pt_combine = np.round(pt_combine / scale * 9)
    pt_combine = pt_combine.where(pt_combine < 9, 9)
    pt_combine = pt_combine.where(pt_combine > -9, -9)
    pt_combine = pt_combine.astype(int)
    pt_combine.to_csv('data/' + name + '.csv')
    if (i+1) % 100 == 0:
        print "finished name", i+1

finished name 100
finished name 200
finished name 300
finished name 400
finished name 500
finished name 600
finished name 700
finished name 800
finished name 900
finished name 1000


## 7. Write the name list to a CSV file as well

This file will be used by the Javascript code to pull in the list of possible names to choose from.

In [13]:
name_by_percent = pd.DataFrame(data={'Name': name_list, 'Percent': percent_list}).set_index('Name')
name_by_percent = name_by_percent.sort()
name_by_percent.head()

,Percent
Name,
Aaliyah,0.481753
Aaron,1.212888
Abby,0.274597
Abigail,1.701651
Abraham,0.161492


In [14]:
name_by_percent.to_csv('namelist.csv')